In [1]:
import os
import random as rn
import numpy as np
import tensorflow as tf

import data_processing as dp
import betaVAE_shapes3d as betaVAE
import get_latents_given_factor as gdist

2022-04-30 14:44:45.675711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/openmpi/lib
2022-04-30 14:44:45.675745: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
sd = 330

os.environ['PYTHONHASHSEED']=str(sd)
rn.seed(sd)
np.random.seed(sd)
tf.random.set_seed(sd)

In [3]:
#import tensorflow_datasets as tfds
#ds = tfds.load(
#        'shapes3d',
#        split='train',
#        data_dir='/share/data1/dpiras',
#        download=True,
#        with_info=True,
#        as_supervised=False,
#        shuffle_files=False
#        )

In [19]:
dict_labels = {'floor_hue': 10,
         'object_hue': 10,
         'orientation': 15,
         'scale': 8,
         'shape': 4,
         'wall_hue': 10}

def preprocess(features):
    image = tf.image.convert_image_dtype(features['image'], tf.float32)
    return image

def extract_labels(features):
    labels_value = []#np.empty(6)
    for i, factor in enumerate(dict_labels.keys()):
        label = 'label_' + factor
        label_value = features[label]
        labels_value.append(label_value)
    return labels_value
    
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_dir = '/share/data1/dpiras'
train_ds, val_ds, test_ds, metadata = dp.DataPreparation(data_dir=data_dir).train_val_test_split('shapes3d', data_dir=data_dir, split_at=[80, 90])
subds = train_ds.take(1).map(preprocess, num_parallel_calls=AUTOTUNE).batch(1) # if this throws error, copy preprocess() from dp file and directly define it in this notebook.

In [7]:
latent_dim = 6
beta =  2
num_trials = 1000
epoch = 60

print(f'beta = {beta}')
tf.keras.backend.clear_session()
model = betaVAE.betaVAE(latent_dim, beta, batch_size=1000)

weights_filename = f'weights-training-ep{epoch}-beta={beta}-zdim={latent_dim}-final.h5'
print('loading weights from: ', weights_filename)
    
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4))
model.evaluate(subds)
model.load_weights(weights_filename)

beta = 2
using seed 330, latent_dim = 6
encoder summary
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 32)        1568      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        16416     
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 64)          32832     
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 64)          65600     
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 


In [8]:
all_test_set = test_ds.take(-1)

In [9]:
test_numpy = np.array(list(all_test_set.map(preprocess, num_parallel_calls=AUTOTUNE).as_numpy_iterator()))
latent_means_and_stds = model.encode(test_numpy)

### Also get the labels

In [21]:
#all_test_set.map(extract_labels)

all_labels = np.array(list(all_test_set.map(extract_labels, num_parallel_calls=AUTOTUNE).as_numpy_iterator()))

In [22]:
all_labels

array([[ 5,  3, 13,  1,  0,  0],
       [ 4,  1, 10,  7,  2,  2],
       [ 9,  0,  3,  5,  3,  9],
       ...,
       [ 5,  6,  0,  6,  0,  6],
       [ 6,  9, 12,  1,  1,  5],
       [ 5,  6,  2,  1,  2,  8]])

In [26]:
latent_means_and_stds[0].numpy().shape

(48000, 6)

In [35]:
means, stds

(array([[-1.3845493 , -1.7926698 ,  0.69978315, -0.10331669,  1.644395  ,
         -0.7225574 ],
        [-0.5640063 ,  1.1521658 ,  1.0989418 ,  0.828569  , -1.0567416 ,
         -1.8896945 ],
        [ 0.72350097, -1.4067838 , -0.36762276, -0.7434684 , -0.7545973 ,
         -0.38959956],
        ...,
        [ 1.7845685 , -0.48801848,  0.6806236 ,  1.7137874 ,  0.39083007,
          0.40228546],
        [-1.0427899 ,  0.7490751 ,  0.4263998 , -0.43666318, -0.40123376,
          0.6686886 ],
        [ 1.0277547 , -0.25283536,  0.6889079 , -0.77352566,  0.39805293,
         -0.13918954]], dtype=float32),
 array([[0.18503612, 0.33128443, 0.04612737, 0.13190652, 0.21126184,
         0.05307809],
        [0.08741339, 0.16218977, 0.07452913, 0.14912441, 0.08058663,
         0.191702  ],
        [0.10864212, 0.07380977, 0.04403514, 0.38181266, 0.04908057,
         0.04745859],
        ...,
        [0.3345761 , 0.5701178 , 0.04483068, 0.28624254, 0.04458302,
         0.04435949],
        [0.

In [36]:
means = latent_means_and_stds[0].numpy()
stds = np.exp(latent_means_and_stds[1].numpy()/2)

#for latent in range(6):
#    samples = np.random.default_rng(42).normal(lmean[:, latent_number], lstd[:, latent_number], len(lmean[:, latent_number]))
z_samples = np.random.default_rng(42).normal(means, stds)

In [37]:
all_labels, z_samples

(array([[ 5,  3, 13,  1,  0,  0],
        [ 4,  1, 10,  7,  2,  2],
        [ 9,  0,  3,  5,  3,  9],
        ...,
        [ 5,  6,  0,  6,  0,  6],
        [ 6,  9, 12,  1,  1,  5],
        [ 5,  6,  2,  1,  2,  8]]),
 array([[-1.32816559, -2.13720032,  0.73439949,  0.02074994,  1.23221571,
         -0.79167463],
        [-0.55283137,  1.10087446,  1.09768963,  0.70135932, -0.98587388,
         -1.74059019],
        [ 0.73067468, -1.3235824 , -0.34703592, -1.07155715, -0.73649881,
         -0.43510677],
        ...,
        [ 1.68768887, -0.74081057,  0.73721287,  2.19284867,  0.4083426 ,
          0.46045534],
        [-1.01627759,  0.82975403,  0.42322734, -0.66943965, -0.33584805,
          0.68945725],
        [ 1.02182295, -0.04000615,  0.74913957, -0.73955026,  0.32550202,
         -0.1896403 ]]))

In [39]:
np.save('./labels.npy', all_labels)
np.save('./latents.npy', z_samples)

# Ignored
# Get list of means and stds for every factor, as well as sampled latents

Returns dicts where the key is the factor that is systematically varied.

Actually I'm not sure if here I'm only using 1000 samples in total... if this is the case you might want to add an extra loop in gdist.get_samples() so you enumerate through the entire test set.

In [ ]:
dict_means = dict()
dict_logvars = dict() # this is the natural log of the variance
dict_samples = dict()

for factor in gdist.shapes3d_factors_dict().keys():
    print(f'factor = {factor}')
    means_ls, logvar_ls, latent_samples_arr = gdist.get_samples(factor, model, test_ds, latent_dim, repeats_per_image_batch=10)
    # above should be returning a list for mean and a list for logvariance, where each element are sampled latents when factor value is fixed to i=1, 2, ...
    # plus an array where one dimension is the number of samples, and the other dimension is the number of latents
    dict_means[factor] = means_ls
    logvar_ls[factor] = logvar_ls
    dict_samples[factor]  = latent_samples_arr
